In [ ]:
# stdlib
import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
use_live_bigquery = False

os.environ["TEST_BIGQUERY_APIS_LIVE"] = str(use_live_bigquery)

# Setup

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party
from apis import make_submit_query
from apis import make_test_query
from helpers import create_user
from helpers import make_user

# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url=SERVER_URL, email="info@openmined.org", password="changethis"
)

In [ ]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

In [ ]:
mock_func = make_test_query(
    settings={
        "rate_limiter_enabled": True,
        "calls_per_min": 10,
    }
)

private_func = make_test_query(
    settings={
        "rate_limiter_enabled": False,
    }
)

new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=new_endpoint)

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_1_col_id = test_settings.get("table_1_col_id", default="table_id")
table_1_col_score = test_settings.get("table_1_col_score", default="colname")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [ ]:
submit_query_function = make_submit_query(
    settings={},
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=submit_query_function)

# Create jobs

In [ ]:
# third party
from job_helpers import TestJob
from job_helpers import create_job_functions
from job_helpers import create_jobs
from job_helpers import extract_code_path

# Test queries

In [ ]:
# stdlib
from typing import Any


def test_query(client: Any, query: str, should_succeed: bool) -> tuple:
    if should_succeed:
        mock_result = client.api.services.bigquery.test_query.mock(sql_query=query)
        private_result = client.api.services.bigquery.test_query.mock(sql_query=query)
    else:
        try:
            mock_result = client.api.services.bigquery.test_query.mock(sql_query=query)
            assert False, f"query should raise, got {mock_result}"
        except Exception as e:
            mock_result = e

        try:
            private_result = client.api.services.bigquery.test_query.private(
                sql_query=query
            )
            assert False, f"query should raise, got {private_result}"
        except Exception as e:
            private_result = e

    return mock_result, private_result

In [ ]:
# Loop through all test queries, test if they behave as expected


results = []

for fn in create_job_functions:
    job = fn(users[0])

    print(f"Querying {job.job_type}, should succeed: {job.should_succeed}")
    mock_result, private_result = test_query(high_client, job.query, job.should_succeed)
    results.append((mock_result, private_result))

In [ ]:
n_per_user = 1

jobs = create_jobs(users, n_per_user=n_per_user)

print(f"num jobs: {len(jobs)}")
for job in jobs[:2]:
    print(f"Job type: {job.job_type}, should succeed: {job.should_succeed}")

assert len(jobs) == len(users) * n_per_user
assert all(isinstance(j, TestJob) for j in jobs)
assert all(job.client is not None for job in jobs)

# Submit jobs


In [ ]:
# stdlib
import time

responses = []

for job in jobs:
    client = job.client

    if not job.should_submit:
        # Submitting should throw error (eg func_name invalid syntax)
        with sy.raises(sy.SyftException):
            response = client.api.services.bigquery.submit_query(
                func_name=job.func_name, query=job.query
            )
        responses.append(None)
    else:
        response = client.api.services.bigquery.submit_query(
            func_name=job.func_name, query=job.query
        )
        job.code_path = extract_code_path(response)
        responses.append(response)

    time.sleep(1)

In [ ]:
for job in jobs:
    print(
        f"Job {job.func_name:.20} is submitted, {job.should_submit}, should be submitted {job.is_submitted}"
    )

assert all(job.is_submitted == job.should_submit for job in jobs)

## Test: cannot execute

In [ ]:
submitted_jobs = [job for job in jobs if job.should_submit]

assert len(submitted_jobs)  # failsafe for next tests

In [ ]:
# Blocking

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = execute_code_fn()

In [ ]:
# Nonblocking

# syft absolute
from syft.service.job.job_stash import JobStatus

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    result_job = execute_code_fn(blocking=False)
    result_job.wait()
    assert isinstance(result_job.result, sy.SyftError)
    assert result_job.status == JobStatus.ERRORED
    time.sleep(1)